In [1]:
import numpy as np
import scipy.special as sysp
import scipy.spatial as syspat
import sys
import pandas as pd
import gc
import matplotlib.pyplot as plt

In [2]:

#///////////////////////////////////////////////////////////////////
# HOD Fits
#///////////////////////////////////////////////////////////////////
class HODFits(object):
    """ Fitting functions and derivatives specifying best fit 5-parameter HOD fits 
         from Paul, Pahwa, Paranjape 2019, Table 3.
    """
    def __init__(self):
        """ Best fit HOD parameters and derivatives as function of luminosity threshold Mr. """

        # dictionary for best fit values of a0,a1,a2 for 5 HOD params
        # lgMmin,siglgM,lgM1,lgM0,alpha
        # fits are of the form a0 + a1*x + a2*x**2 for all except siglgM
        # in which case fit is a0 + a1*erf(x/a2)

        self.bf = {'lgMmin':np.array([12.33,-0.85,0.19]),
                   'siglgM':np.array([0.44,-0.16,0.3]),
                   'lgM1':np.array([13.52,-0.72,0.16]),
                   'lgM0':np.array([12.24,-0.54,0.0]),
                   'alpha':np.array([1.16,-0.20,0.10])}

    def hod_fit(self,param,Mr):
        """ Return HOD parameter param (string) at luminosity threshold Mr.
             param can be one of 'lgMmin','siglgM','lgM1','lgM0','alpha'.
        """
        a0,a1,a2 = self.bf[param]
        x = Mr + 20.5
        out = (a0 + a1*x + a2*x**2
               if param != 'siglgM' else
               a0 + a1*sysp.erf(x/a2))
        return out

    def hod_fit_deriv(self,param,Mr):
        """ Return derivative wrt Mr for HOD parameter param (string) at luminosity threshold Mr.
             param can be one of 'lgMmin','siglgM','lgM1','lgM0','alpha'.
        """
        a0,a1,a2 = self.bf[param]
        x = Mr + 20.5
        out = (a1 + 2*a2*x
               if param != 'siglgM' else
               (a1/a2)*2/np.sqrt(np.pi)*np.exp(-(x/a2)**2))
        return out

    

#///////////////////////////////////////////////////////////////////
# HOD Functions
#///////////////////////////////////////////////////////////////////
class HODFunctions(HODFits):
    """ Functions and derivatives specifying 5-parameter HODs 
         Uses fits from Paul, Pahwa, Paranjape 2019.
    """
    def __init__(self):
        """ HOD functions and derivatives as function of luminosity threshold Mr. """

        HODFits.__init__(self)

    def erf_arg(self,Mr,lgm):
        """ Convenience function.
             If both arguments are arrays, output has shape (lgm.size,Mr.size).
        """
        Mr_sc = np.isscalar(Mr)
        lgm_sc = np.isscalar(lgm)

        lgm_arr = np.outer(lgm,np.ones(Mr.size)) if ((not Mr_sc) & (not lgm_sc)) else 1.0*lgm
        erfarg = (lgm_arr - self.hod_fit('lgMmin',Mr))/self.hod_fit('siglgM',Mr)
        return erfarg

    def fcen_thresh(self,Mr,lgm):
        """ fcen(>L|m) = (1/2)[1 + erf(log10(m/Mmin)/siglgM)]. 
             If both arguments are arrays, output has shape (lgm.size,Mr.size).
        """
        erfarg = self.erf_arg(Mr,lgm)
        out = 0.5*(1+sysp.erf(erfarg))
        return out

    def Nsat_thresh(self,Mr,lgm):
        """ Nsat(>L|m) = ((m - M0)/M1)^alpha.
             If both arguments are arrays, output has shape (lgm.size,Mr.size).
        """
        M0 = 10**self.hod_fit('lgM0',Mr)
        M1 = 10**self.hod_fit('lgM1',Mr)
        alpha = self.hod_fit('alpha',Mr)
        mhalo = 10**lgm

        Mr_sc = np.isscalar(Mr)
        lgm_sc = np.isscalar(lgm)

        if Mr_sc & lgm_sc:
            out = ((mhalo - M0)/M1)**alpha if mhalo > M0 else 0.0
        else:
            mhalo_arr = np.outer(mhalo,np.ones(Mr.size)) if ((not Mr_sc) & (not lgm_sc)) else 1.0*mhalo
            Dm = (mhalo_arr - M0)/M1
            Dm[Dm < 0.0] = 0.0
            out = Dm**alpha

        return out
    
    

In [3]:
class Mocker(HODFunctions):
    def __init__(self):
        
        HODFunctions.__init__(self)
        self.halo_cat = 'D:/out_1.parents'
        self.redshift = 0.0
        self.Om = 0.276
        self.Lbox = 300.0 # Mpc/h
        self.massdef = 'm200b'
        self.mmin = 2e11 # Msun/h
        self.Mrmax = -20.5           #sectorfx
        self.Dvir = 200*self.Om
        
        self.rhoc = 2.7754e11 # (Msun/h) / (Mpc/h)^3
        self.rhoc_z = self.rhoc*self.EHub(self.redshift)**2
        
        self.rng = np.random.RandomState(seed=42)

        dMr = 0.001
        nMr = int((23.5+self.Mrmax)/dMr)
        self.Mrvals = np.linspace(-23.5,self.Mrmax,nMr) # useful for interpolation later
        self.dMr = self.Mrvals[1]-self.Mrvals[0]
        
        self.halodatatype = {'ID':int,'descID':int,
                            'mbnd_vir':float,'vmax':float,'vrms':float,'rvir':float,'rs':float,'np':int,
                            'x':float,'y':float,'z':float,'vx':float,'vy':float,'vz':float,
                            'Jx':float,'Jy':float,'Jz':float,'spin':float,'rs_klypin':float,
                            'mvir':float,'m200b':float,'m200c':float,'mCustom2':float,'mCustom':float,
                            'Xoff':float,'Voff':float,'spin_bullock':float,'b_to_a':float,'c_to_a':float,
                            'Ax':float,'Ay':float,'Az':float,'b_to_a_500c':float,'c_to_a_500c':float,
                            'Ax_500c':float,'Ay_500c':float,'Az_500c':float,
                            'TbyU':float,'Mpe_Behroozi':float,'Mpe_Diemer':float,'halfmassradius':float,
                            'pid':int} 

        
        self.halodatanames = ['ID','descID',
                              'mbnd_vir','vmax','vrms','rvir','rs','np',
                              'x','y','z','vx','vy','vz','Jx','Jy','Jz','spin','rs_klypin',
                              'mvir','m200b','m200c','mCustom2','mCustom',
                              'Xoff','Voff','spin_bullock','b_to_a','c_to_a',
                              'Ax','Ay','Az','b_to_a_500c','c_to_a_500c',
                              'Ax_500c','Ay_500c','Az_500c',
                              'TbyU','Mpe_Behroozi','Mpe_Diemer','halfmassradius',
                              'pid']
        
    ############################################################
    def EHub(self,z):
        return np.sqrt(self.Om*(1+z)**3 + 1-self.Om)
    ############################################################
    
    ############################################################
    def read_this(self):
        halos = pd.read_csv(self.halo_cat,dtype=self.halodatatype,names=self.halodatatype.keys(),#self.halodatanames,
                            comment='#',delim_whitespace=True,header=None).to_records()
        return halos
    ############################################################
    

    ############################################################
    def prep_halo_data(self):
        """ Reads and cleans catalog by selecting parent objects with mass at least mmin.
            Returns structured array for full halo properties.
        """ 
        halos = self.read_this()
        Nhalos_all = halos.size
        cond_clean = (halos[self.massdef] >= self.mmin)
        cond_clean = cond_clean & (halos['pid'] == -1)

        halos = halos[cond_clean]
        print("Kept {0:d} of {1:d} objects in catalog".format(halos.size,Nhalos_all))

        del cond_clean
        gc.collect()

        return halos
    ############################################################


    ############################################################
    def occupy_halos(self,halo_mass):
        """ Decide which halos are occupied. 
             Input is 1-d array of halo masses. Output is 1-d boolean array of same size.
        """
        print("Occupying halos... ")

        occupy = np.ones(halo_mass.size,dtype=bool)
        fcen_min = self.fcen_thresh(self.Mrmax,np.log10(halo_mass))
        u = self.rng.rand(halo_mass.size)
        occupy[u > fcen_min] = False

        print_string = "... {0:d} of {1:d} halos occupied\n".format(np.where(occupy)[0].size,occupy.size)
        print_string += "--------------------------------"
        print(print_string)

        return occupy
    ############################################################
    

    ############################################################
    def assign_centrals(self,halos):
        """ Allocate memory and assign (some) central properties. """

        print("Assigning values for centrals... ")
        ####################################################
        # Allocate memory for centrals
        centrals = np.zeros(halos.size,dtype=[('haloid','int'),
                                              ('lgm','double'),('rvir','double'),('con','double'),
                                              ('x','double'),('y','double'),('z','double'),
                                              ('Mr','double'),('Nsat','int')])

        print("... halo IDs, halo masses, positions")
        centrals['haloid'] = halos['ID']
        centrals['lgm'] = np.log10(halos[self.massdef])
        centrals['x'] = halos['x']
        centrals['y'] = halos['y']
        centrals['z'] = halos['z']
        
        rvir = (3*halos[self.massdef]/(4*np.pi*self.Dvir*self.rhoc_z))**(1/3.)
        # Units Mpc/h [physical]
        centrals['rvir'] = rvir*(1+self.redshift) # store comoving value
        centrals['con'] = rvir/(halos['rs']*1e-3/(1+self.redshift)) # rs is comoving kpc/h
        
        ####################################################
        # Assign central luminosity
        centrals['Mr'] = self.assign_central_luminosity(centrals['lgm'])

        ####################################################
        # Poisson numbers for Nsat
        centrals['Nsat'] = self.get_Poisson(centrals['lgm'])

        print_string = '... done\n'
        print_string += "--------------------------------"
        print(print_string)

        return centrals
    ############################################################


    ############################################################
    def get_Poisson(self,lgm_halos):
        """ Calculate mean Nsat and generate Poisson numbers of satellites. 
            Can be generalised to depend on more than halo mass.
        """
        print("... Poisson satellite counts")
        mean_Nsat = self.Nsat_thresh(self.Mrmax,lgm_halos)
        Nsat_values = self.rng.poisson(mean_Nsat)

        return Nsat_values
    ############################################################
    

    ############################################################
    def assign_central_luminosity(self,lgm_halos):
        """ Assign central luminosities. Can be generalised to depend on more than halo mass. """
        
        print("... luminosities")
        Mr_cen = np.zeros(lgm_halos.size,dtype=float)

        ############################################
        # Generate uniform deviates
        u = self.rng.rand(lgm_halos.size)

        ################################################
        for h in range(lgm_halos.size):
            # Luminosity function is Pcen(>L|m) = fcen(>L|m)/fcen(>Lmin|m)
            PcenL = self.fcen_thresh(self.Mrvals,lgm_halos[h])/self.fcen_thresh(self.Mrmax,lgm_halos[h])
            # keeping this in loop is not much slower than vectorised calc, but more memory-efficient

            # Find largest L (smallest Mr) where Pcen(>L|m) > u
            Mr_upp = self.Mrvals[PcenL > u[h]][0]

            # Find smallest L (largest Mr) where Pcen(>L|m) <= u
            Mr_low = self.Mrvals[PcenL <= u[h]][-1]
            # choose fainter of the two (enforce nesting of HOD)
            Mr_cen[h] = np.max([Mr_upp,Mr_low])
        ################################################
        
        del u
        gc.collect()

        return Mr_cen
    ############################################################

    

    ########################################################
    def assign_satellites(self,centrals):
        """ Allocate memory and assign (some) satellite values."""
        print("Assigning values for satellites... ")

        Nsat_tot = np.sum(centrals['Nsat'])

        ####################################################
        # Allocate memory for satellites
        satellites = np.zeros(Nsat_tot,dtype=[('haloid','int'),
                                              ('lgm','double'),('con','double'),
                                              ('x','double'),('y','double'),('z','double'),
                                              ('Mr','double')])

        print("... halo properties")
        s_lo = 0
        for h in range(centrals.size):
            s_hi = s_lo + centrals['Nsat'][h]
            ################################################
            if centrals['Nsat'][h]:
                sl = np.s_[s_lo:s_hi]
                ############################################
                satellites['haloid'][sl] = centrals['haloid'][h]
                satellites['lgm'][sl] = centrals['lgm'][h]
                satellites['con'][sl] = centrals['con'][h]
            ################################################
            # Update
            s_lo = s_hi

        print("... positions")
        s_lo = 0
        for h in range(centrals.size):
            s_hi = s_lo + centrals['Nsat'][h]
            ################################################
            if centrals['Nsat'][h]:
                sl = np.s_[s_lo:s_hi]
                ############################################
                sx,sy,sz = self.assign_satellite_pos(centrals,h)
                satellites['x'][sl] = sx
                satellites['y'][sl] = sy
                satellites['z'][sl] = sz
            ################################################
            # Update
            s_lo = s_hi

        print("... luminosities")
        s_lo = 0
        for h in range(centrals.size):
            s_hi = s_lo + centrals['Nsat'][h]
            ################################################
            if centrals['Nsat'][h]:
                ############################################
                sMr = self.assign_satellite_luminosities(centrals,h)
                satellites['Mr'][s_lo:s_hi] = sMr
            ################################################
            # Update
            s_lo = s_hi

        gc.collect

        print_string = '... done\n'
        print_string += "--------------------------------"
        print(print_string)

        return satellites
    ########################################################
    
    
    ########################################################
    def assign_satellite_luminosities(self,centrals,h):
        """ Assign satellite luminosities. """

        Nsat_h = centrals['Nsat'][h]
        Mr_cen = centrals['Mr'][h]
        lgmhalo = centrals['lgm'][h]
        PsatL = self.Nsat_thresh(self.Mrvals,lgmhalo)/self.Nsat_thresh(self.Mrmax,lgmhalo)

        ####################################################
        # Generate luminosities, without worrying about Lcen.
        # ... generate Nsat_h uniform deviates
        u = self.rng.rand(Nsat_h)
        # ... find largest L (smallest Mr) where Psat(>L|m) > u
        Mr_sat = np.array([self.Mrvals[(PsatL >= u[s])][0] for s in range(Nsat_h)])

        return Mr_sat
    ########################################################
    

    ########################################################
    def assign_satellite_pos(self,centrals,h):
        """ Assign satellite positions and velocities acc to NFW profile using halo concentration. """
        # Number of satellites in this halo
        Nsat_h = centrals['Nsat'][h]
        # Virial radius [comoving Mpc/h]
        rvir = centrals['rvir'][h]
        con = centrals['con'][h]

        sat_pos = self.gen_NFW_profile(Nsat_h,cvir=con,Rvir=rvir,rng=self.rng)
        sx = sat_pos[:,0] + centrals['x'][h]
        sy = sat_pos[:,1] + centrals['y'][h]
        sz = sat_pos[:,2] + centrals['z'][h]

        sx = sx % self.Lbox
        sy = sy % self.Lbox
        sz = sz % self.Lbox

        return sx,sy,sz
    
    
    ############################################################
    def gen_NFW_profile(self,Nsat,cvir=None,Rvir=None,rng=None,include_cen=False,clean_up=False):
        """ Generate 3-d points distributed according to specified radial profile.
            Returns array of shape (Ngal,3) where Ngal = Nsat if include_cen = False else Nsat+1
            and central is assumed to be at origin. (Can include spatial offset later.)
        """
        if rng is None:
            rng = np.random.RandomState(42)
        phi = 2*np.pi*rng.rand(Nsat)
        cos_theta = 2*rng.rand(Nsat) - 1
        sin_theta = np.sqrt(1-cos_theta**2)
        
        rsamp = self.gen_rsamp(Nsat,cvir=cvir,Rvir=Rvir,rng=rng,clean_up=clean_up)

        x_trc = rsamp*sin_theta*np.sin(phi)
        y_trc = rsamp*sin_theta*np.cos(phi)
        z_trc = rsamp*cos_theta

        sample = np.array([x_trc,y_trc,z_trc]).T
        if include_cen:
            sample = np.append(sample,[[0.,0.,0.]],axis=0)
            
        if clean_up:
            del x_trc,y_trc,z_trc
            gc.collect()

        return sample
    ############################################################
    
    
    ############################################################
    def gen_rsamp(self,Nsat,cvir=None,Rvir=None,rng=None,clean_up=False):
        """ Generate radial location sample for speficied profile. 
            Return array of r values of shape (Nsat,).
        """
        xmax = 2*cvir

        if rng is None:
            rng = np.random.RandomState(42)

        xfine = np.linspace(0,xmax,100000)
        dx = xfine[1]-xfine[0]
        Px = np.zeros_like(xfine)
        
        rs = Rvir/cvir
        Px = self.Mencl_nfw(xfine)
        ind_small = np.where(xfine < 1e-3)[0]
        if ind_small.size:
            Px[ind_small] = xfine[ind_small]**2/2 - 2*xfine[ind_small]**3/3.0 
            Px[ind_small] = Px[ind_small] + 3*xfine[ind_small]**4/4.0 - 4*xfine[ind_small]**5/5.0
        Px /= self.Mencl_nfw(cvir)
                
        vran = rng.rand(Nsat)
        rsamp = np.interp(vran,Px,xfine)
        if np.any(rsamp <= dx):
            print('! Warning ! : Ignore separations x < {0:.2e}'.format(dx))

        if clean_up:
            del xfine,Px
            gc.collect()
        
        rsamp *= rs
        
        return rsamp
    ############################################################
    
    
    ############################################################
    def Mencl_nfw(self,x):
        """ Convenience function for NFW enclosed mass. Expect x = r/rs. """
        return np.log(1+x) - x/(1+x)
    ############################################################
    
    

    ############################################################
    def mock_it(self):
        """ Main execution routine. """        
        # halo data
        halos_all = self.prep_halo_data()

        # occupy halos
        occupy = self.occupy_halos(halos_all[self.massdef])
        halos = halos_all[occupy]
        all_masses = halos_all[self.massdef]
        del halos_all
        gc.collect()

        # galaxy properties except color and derived props
        centrals = self.assign_centrals(halos)
        satellites = self.assign_satellites(centrals)
        del halos
        gc.collect()

        print_string = "... {0:d} galaxies created\n".format(centrals.size + satellites.size)
        print_string += "--------------------------------"
        print(print_string)

        print_string = '... all done\n'
        print_string += "--------------------------------"
        print(print_string)

        return centrals,satellites,all_masses

    ############################################################
    


In [4]:
mm = Mocker()

In [5]:
# halos = mm.prep_halo_data()
# occupy = mm.occupy_halos(halos[mm.massdef])

In [6]:
# plt.yscale('log')
# plt.hist(np.log10(halos['m200b']),bins=100,label='all')
# plt.hist(np.log10(halos['m200b'][occupy]),bins=100,label='occupied')
# plt.legend()
# plt.show()

In [7]:
centrals,satellites,all_masses = mm.mock_it()

PermissionError: [Errno 13] Permission denied: 'D:/out_1.parents'

In [ ]:
plt.yscale('log')
plt.xlim(mm.Mrmax+0.1,-23.5)
plt.hist(centrals['Mr'],bins=50,label='centrals')
plt.hist(satellites['Mr'],bins=50,label='satellites')
plt.legend()
plt.show()

In [ ]:
class TwoPointCorrelationFunctionPeriodic(object):
    """ 2-point (cross-)correlation functions in bins of separation. 
        Assumes complete, cubic periodic box throughout.
    """
    #############################################################
    def __init__(self,lgbin=1,rmin=1e-2,rmax=3e1,nbin=15,Lbox=300.0,proj=0,pimax=60.0):
        """ Initialise the following:
            -- lgbin      : use logarithmic (1) or linear (0) binning
            -- rmin,rmax: min,max separations in r (or rp)
            -- nbin      : number of bins in r (or rp).
            -- Lbox     : Box size in same units as rmin,rmax,pimax.
            -- proj       : if 1, calculate projected correlation function wp(rp), else xi(r)
            -- pimax    : max separation in pi (bins in pi will be rp-dependent)
             
            Methods: 
            pair_counts,DD_only,RR_theory,twoPCF
            """
        self.lgbin = lgbin
        self.rmin = rmin
        self.rmax = rmax
        self.nbin = nbin
        self.Lbox = Lbox
        self.proj = proj
        self.pimax = pimax

        self.TINY = 1e-15
        self.N_SPLIT = 8000 # max number of objects that can be handled on 1 cpu

        print_string = "Initialising 2pcf calculator for periodic boxes\n"
        print_string += "--------------------------------"
        print(print_string)

        if self.proj:
            print("... projected 2pcf calculated")
            if self.lgbin==1:
                self.rpbin = np.logspace(np.log10(self.rmin),np.log10(self.rmax),self.nbin+1)
                self.rpmid = np.sqrt(self.rpbin[1:]*self.rpbin[:-1])
            else:
                self.rpbin = np.linspace(self.rmin,self.rmax,self.nbin+1)
                self.rpmid = 0.5*(self.rpbin[1:]+self.rpbin[:-1])
            self.NR = 50
            self.NR_INT = self.NR*3 # seems converged at NR=50,NR_INT=50*3 or dlnr=0.13,dlnr_int=0.044
            self.RMIN = self.rmin
            self.RMAX = 0.5*self.Lbox # NOTE THIS
            self.rbin = np.logspace(np.log10(self.RMIN),np.log10(self.RMAX),self.NR+1)
            self.rmid = np.sqrt(self.rbin[1:]*self.rbin[:-1])
            self.rbin_int = np.logspace(np.log10(self.RMIN),np.log10(self.RMAX),self.NR_INT+1)
            self.rmid_int = np.sqrt(self.rbin_int[1:]*self.rbin_int[:-1])
            self.dlnr_int = np.log(self.rmid_int[1]/self.rmid_int[0])
        else:
            print("... monopole 2pcf calculated")
            if self.lgbin==1:
                self.rbin = np.logspace(np.log10(self.rmin),np.log10(self.rmax),self.nbin+1)
                self.rmid = np.sqrt(self.rbin[1:]*self.rbin[:-1])
            else:
                self.rbin = np.linspace(self.rmin,self.rmax,self.nbin+1)
                self.rmid = 0.5*(self.rbin[1:]+self.rbin[:-1])

        print_string = "... initialisation complete\n"
        print_string += "--------------------------------"
        print(print_string)

    #############################################################


    #############################################################
    def pair_counts(self,pos_1,pos_2):
        """ Unnormalised binned pair counts 
            between two data sets (can be the same data set).
            pos_j should have shape (ndata_j,3) and
            Returns vector of length self.nbin. 
        """
        tree_1 = syspat.cKDTree(pos_1,boxsize=self.Lbox)
        tree_2 = syspat.cKDTree(pos_2,boxsize=self.Lbox)

        cum_counts = tree_1.count_neighbors(tree_2,self.rbin,cumulative=True)
        bin_counts = np.diff(cum_counts)

        del tree_1,tree_2
        gc.collect()
        ###################

        return bin_counts 
    #############################################################



    #############################################################
    def RR_theory(self):
        print("... ... calculating RR from theory")
        # valid for rmax < Lbox/2, else see Deserno 04
        return 4*np.pi/3*(self.rbin[1:]**3-self.rbin[:-1]**3)/self.Lbox**3
    #############################################################


    #############################################################
    def twoPCF(self,pos_data,pos_data2=None):
        """ Auto/cross-correlation of data points.
            Assumes pos_data.shape() = (ndata,3).
            Applies recursive binary split for large data sets.
            Returns Peebles-Hauser estimator DD/RR - 1.
        """
        if pos_data.shape[1] != 3:
            raise TypeError("Incompatible data shape. Expected (ndata,3).")

        DD = self.DD_split(pos_data,pos_data2=pos_data2)

        RR = self.RR_theory()
        cf_r = DD/(RR + self.TINY) - 1.0    

        if self.proj:
            print("... ... projecting")
            cf = np.zeros(self.nbin,dtype=float)
            cf_r_int = np.interp(self.rmid_int,self.rmid,cf_r)
            for rp in range(self.nbin):
                cond = (self.rmid_int > self.rpmid[rp]) & (self.rmid_int**2 < (self.rpmid[rp]**2 + self.pimax**2))
                ind = np.where(cond)[0]
                if ind.size:
                    Drp = self.rmid_int[ind[0]] - self.rpmid[rp]
                    add_this = np.sqrt(2*Drp/self.rpmid[rp])*cf_r_int[ind[0]]*self.rpmid[rp]
                    cf[rp] = add_this + 2*np.trapz((self.rmid_int[cond]*cf_r_int[cond]
                                                    /np.sqrt(self.rmid_int[cond]**2 - self.rpmid[rp]**2)),
                                                   x=self.rmid_int[cond])
        else:
            cf = 1.0*cf_r

        print("... ... done")
        
        return cf
    #############################################################


    #############################################################
    def DD_only(self,pos_data1,pos_data2=None):
        """ DD (or D1D2) calculation.
            Assumes pos_data1.shape() = (ndata,3).
            If pos_data2 is not None, assumes pos_data2.shape() = (ndata2,3).
            Returns DD or D1D2.
        """
        ndata1 = pos_data1.shape[0]
        if pos_data2 is None:
            cf = 1.0*self.pair_counts(pos_data1,pos_data1)/ndata1/(ndata1-1)
        else:
            ndata2 = pos_data2.shape[0]
            cf = 1.0*self.pair_counts(pos_data1,pos_data2)/ndata1/ndata2

        return cf
    #############################################################


    #############################################################
    def DD_split(self,pos_data,pos_data2=None):
        """ DD calculation wrapper.
            Assumes pos_data.shape() = (ndata,3).
            If pos_data2 is not None, assumes pos_data2.shape() = (ndata2,3).
            Returns DD (or D1D2), applying binary recursive split for large data sets.
        """
        ndata = pos_data.shape[0]
        ndata_by_2 = ndata//2
        if ndata < self.N_SPLIT:
            print_string = '... ... calculating '
            print_string += "DD" if pos_data2 is None else "D1D2"
            print(print_string)
            cf = self.DD_only(pos_data,pos_data2=pos_data2)
        else:
            print("... ... binary split")
            if pos_data2 is None:
                pd1s = pos_data[:ndata_by_2].copy()
                ndata1s = pd1s.shape[0]
                pd1spr = pos_data[ndata_by_2:].copy()
                ndata1spr = pd1spr.shape[0]
                D1D1 = self.DD_split(pd1s)
                D1D1pr = self.DD_split(pd1s,pos_data2=pd1spr)
                D1prD1pr = self.DD_split(pd1spr)
                cf = ndata1s*(ndata1s-1)*D1D1 + 2*ndata1s*ndata1spr*D1D1pr + ndata1spr*(ndata1spr-1)*D1prD1pr
                cf = 1.0*cf/ndata/(ndata-1)
                del pd1s,pd1spr
                gc.collect()
            else:
                ndata2 = pos_data2.shape[0]
                ndata2_by_2 = ndata//2
                pd1s = pos_data[:ndata_by_2].copy()
                ndata1s = pd1s.shape[0]
                pd1spr = pos_data[ndata_by_2:].copy()
                ndata1spr = pd1spr.shape[0]
                pd2s = pos_data2[:ndata2_by_2].copy()
                ndata2s = pd2s.shape[0]
                pd2spr = pos_data2[ndata2_by_2:].copy()
                ndata2spr = pd2spr.shape[0]
                D1D2 = self.DD_split(pd1s,pos_data2=pd2s)
                D1D2pr = self.DD_split(pd1s,pos_data2=pd2spr)
                D1prD2 = self.DD_split(pd1spr,pos_data2=pd2s)
                D1prD2pr = self.DD_split(pd1spr,pos_data2=pd2spr)
                cf = ndata1s*ndata2s*D1D2 + ndata1s*ndata2spr*D1D2pr 
                cf = cf + ndata1spr*ndata2s*D1prD2 + ndata1spr*ndata2spr*D1prD2pr
                cf = 1.0*cf/ndata/ndata2
                del pd1s,pd1spr,pd2s,pd2spr
                gc.collect()

        return cf
    #############################################################


In [ ]:
tpcf_real = TwoPointCorrelationFunctionPeriodic(rmin=0.01,rmax=60.,nbin=20,Lbox=mm.Lbox,proj=0)
tpcf = TwoPointCorrelationFunctionPeriodic(rmin=0.1,rmax=40.,nbin=20,Lbox=mm.Lbox,proj=1,pimax=40.0)

In [ ]:
cpos = np.array([centrals['x'],centrals['y'],centrals['z']]).T
spos = np.array([satellites['x'],satellites['y'],satellites['z']]).T
gpos = np.vstack((cpos,spos))

In [ ]:
# print('Centrals auto')
# xir_cen = tpcf_real.twoPCF(cpos,cpos)
# print('Satellites auto')
# xir_sat = tpcf_real.twoPCF(spos,spos)
# print('All auto')
# xir_gal = tpcf_real.twoPCF(gpos,gpos)

In [ ]:
# plt.xscale('log')
# plt.yscale('log')
# plt.xlabel("$r \\, (h^{{-1}}{{\\rm Mpc}})$")
# plt.ylabel("$\\xi(r)$")
# plt.plot(tpcf_real.rmid,xir_cen,'b-',label='centrals')
# plt.plot(tpcf_real.rmid,xir_sat,'r-',label='satellites')
# plt.plot(tpcf_real.rmid,xir_gal,'k-',label='all')
# plt.legend()
# plt.show()

In [ ]:
# zehavi = np.loadtxt('data/zehavi_data_dex_separated.txt').T
# z11_rp = zehavi[0]
# z11_wp_20p5 = zehavi[7]
# z11_errwp_20p5 = zehavi[8]

In [ ]:
print('Centrals auto')
wp_cen = tpcf.twoPCF(cpos,cpos)
print('Satellites auto')
wp_sat = tpcf.twoPCF(spos,spos)
print('All auto')
wp_gal = tpcf.twoPCF(gpos,gpos)

In [ ]:
plt.xscale('log')
plt.yscale('log')
plt.xlabel("$r_{{\\rm p}} \\, (h^{{-1}}{{\\rm Mpc}})$")
plt.ylabel("$w_{{\\rm p}}(r_{{\\rm p}}) \\, (h^{{-1}}{{\\rm Mpc}})$")
plt.plot(tpcf.rpmid,wp_cen,'b-',label='centrals')
# plt.plot(tpcf.rpmid,wp_sat,'g-',label='satellites')
plt.plot(tpcf.rpmid,wp_gal,'k-',label='cen + sat')
# plt.errorbar(z11_rp,z11_wp_20p5,yerr=z11_errwp_20p5,ls='none',
#              c='crimson',capsize=5,marker='o',label='Zehavi+11')
plt.legend()
# plt.savefig('fig-compareZ11wp-20p5.png',bbox_inches='tight')
plt.show()